In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact
import ipywidgets as widgets

In [4]:
@interact
def interaktif(indeks=0):
    # parameter model sintetik (koordinat hiposenter)
    x0 = 1300
    y0 = 2050
    z0 = -1000

    # parameter model sintetik (Vp dan T0)
    t0 = 0
    vp = 5.1 # (km/s)

    # data koordinat titik stasiun
    xs = np.array([700, 1250, 1200, 1350])
    ys = np.array([2300, 2000, 1700, 1500])
    zs = np.zeros(4)

    # fungsi menghitung travel time
    def travel_time(xhipo, yhipo, zhipo, xsi, ysi, zsi):
        return t0 + (np.sqrt((xsi-xhipo)**2 + (ysi-yhipo)**2 + (zsi-zhipo)**2)/(vp))

    # membuat t observasi
    np.random.seed(10)
    noise = np.random.normal(1, 0.4, len(xs))
    t_obs = travel_time(x0, y0, z0, xs, ys, zs) + (noise)
    
    # model referensi
    m_ref = [700, 1500, -700] # [x0,y0,z0]

    iterasi = indeks
    misfit = np.array([np.inf])
    for i in range(iterasi):

        # forward modelling
        t_fm = travel_time(m_ref[0], m_ref[1], m_ref[2], xs, ys, zs)
        delta_t = t_obs - t_fm

        # menghitung error
        rms = np.sqrt(np.mean((delta_t)**2))
        misfit = np.append(misfit, rms)

        if np.isclose(misfit[-1], misfit[i]) or misfit[i] == 0:
            break
        else:
            # matrix jacobi
            dtdx0 = (m_ref[0]-xs)/(vp*np.sqrt((m_ref[0]-xs)**2 + (m_ref[1]-ys)**2 + (m_ref[2]-zs)**2))
            dtdy0 = (m_ref[1]-ys)/(vp*np.sqrt((m_ref[0]-xs)**2 + (m_ref[1]-ys)**2 + (m_ref[2]-zs)**2))
            dtdz0 = (m_ref[2]-zs)/(vp*np.sqrt((m_ref[0]-xs)**2 + (m_ref[1]-ys)**2 + (m_ref[2]-zs)**2))
            J = np.column_stack([dtdx0, dtdy0, dtdz0])

            # inversi
            m_calc = np.linalg.inv((J.T).dot(J)).dot(J.T).dot(delta_t)
            m_ref = m_calc + m_ref
            #print(m_ref)

    fig = plt.figure(figsize=(9,9))
    ax = plt.axes(projection='3d')
    ax.scatter(x0, y0, z0, label="data observasi", color="red", marker="*", s=100)
    ax.scatter(m_ref[0], m_ref[1], m_ref[2], label="data kalkulasi", color="blue", s=70)
    ax.scatter(xs[0], ys[0], zs[0], color="black", marker="v", s=100, label="stasiun")
    ax.scatter(xs[1], ys[1], zs[1], color="black", marker="v", s=100)
    ax.scatter(xs[2], ys[2], zs[2], color="black", marker="v", s=100)
    ax.scatter(xs[3], ys[3], zs[3], color="black", marker="v", s=100)
    
    ax.text(700,1500, 300, f"iterasi: {indeks}")
    ax.text(700,1500, 200, f"RMS: {np.round(misfit[indeks],2)}")
    ax.legend()
    
    ax.set_xlabel("x (meter)")
    ax.set_ylabel("y (meter)")
    ax.set_zlabel("z (meter)")
    ax.set_xlim([650, 1550])
    ax.set_ylim([1450, 2250])
    ax.set_zlim([-1500, 0])
    ax.view_init(20, 20) 
    plt.show()

interactive(children=(IntSlider(value=0, description='indeks', max=1), Output()), _dom_classes=('widget-intera…

In [5]:
interact(interaktif, indeks=widgets.Play(min=0,max=6, interval=1000))
plt.show()

interactive(children=(Play(value=0, description='indeks', interval=1000, max=6), Output()), _dom_classes=('wid…